## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad, descripcion_oferta=None):
        self.codigo = codigo
        self.nombre = nombre
        self._precio = precio
        self.tipo = tipo
        self._cantidad = cantidad
        self.descripcion_oferta = descripcion_oferta if descripcion_oferta else ""

    def valorTotal(self):
        return self.cantidad * self.precio

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if valor < 0:
            print(f"el precio no puede ser negativo ({valor})")
        else:
            self._precio = valor

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if valor < 0:
            print(f"la cantidad no puede ser negativa ({valor})")
        else:
            self._cantidad = valor


In [2]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

el precio no puede ser negativo (-200)
la cantidad no puede ser negativa (-5)
Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [3]:
class Oferta:
    def __init__(self, descripcion, codigos=None, tipos=None):
        self.descripcion = descripcion
        self.codigos = codigos if codigos is not None else []  
        self.tipos = tipos if tipos is not None else []        

class OfertaDescuento(Oferta):
    def __init__(self, descuento, *args, **kwargs):
        super().__init__(f"Descuento {descuento}%", *args, **kwargs)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if producto.codigo in self.codigos or producto.tipo in self.tipos:
            descuento_total = producto.precio * cantidad * (self.descuento / 100)
            
            return descuento_total  
        return 0

class Oferta2x1(Oferta):
    def __init__(self, *args, **kwargs):
        super().__init__("Oferta 2x1", *args, **kwargs)

    def calcularDescuento(self, producto, cantidad):
        if producto.codigo in self.codigos or producto.tipo in self.tipos:
            producto_oferta = cantidad // 2
            descuento_total = producto_oferta * producto.precio
            
            return descuento_total
        return 0

In [4]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [5]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    @classmethod
    def leer(cls, archivo):
        catalogo = cls()
        with open(archivo, "r") as csvfile:
            leer = csv.DictReader(csvfile)
            for row in leer:
                producto = Producto(row['codigo'], row['nombre'], float(row['precio']), row['tipo'], int(row['cantidad']))
                catalogo.productos.append(producto)
            
        return catalogo
    def guardar(self, archivo):
        with open(archivo, mode='w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['codigo', 'nombre', 'precio', 'tipo', 'cantidad']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for producto in self.productos:
                writer.writerow({
                    'codigo': producto.codigo,
                    'nombre': producto.nombre,
                    'precio': producto.precio,
                    'tipo': producto.tipo,
                    'cantidad': producto.cantidad
                })

    def agregar(self, *productos): 
        for producto in productos:
            self.productos.append(producto)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def obtenerDescripcionOferta(self, producto, cantidad):
        for oferta in self.ofertas:
            descuento = oferta.calcularDescuento(producto, cantidad)
            if descuento > 0:
                return oferta.descripcion 
        return "Descuento 10%"

    def calcularDescuento(self, producto, cantidad):
        descuento_total = 0
        descripcion = "Descuento 10%"
        for oferta in self.ofertas:
            descuento = oferta.calcularDescuento(producto, cantidad)
            if descuento > 0:
                descuento_total += descuento
                descripcion = oferta.descripcion
                break
        return descuento_total
        

    def vender(self, producto, cantidad):
        for p in self.productos:
            if p.codigo == producto.codigo and p.cantidad >= cantidad:
                p.cantidad -= cantidad
                break

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos)

    @property
    def valorTotal(self):
        return sum(producto.valorTotal() for producto in self.productos)

    @property
    def precio_prom(self):
        if self.cantidadProductos == 0:
            return 0
        return self.valorTotal / self.cantidadProductos

    def informe(self):
        informe_str = "INFORME CATALOGO\n"
        informe_str += f"Cantidad de productos:   {self.cantidadProductos}\n"
        informe_str += f"Cantidad de unidades:    {self.cantidadUnidades}\n"
        informe_str += f"Precio Promedio:       $ {self.precio_prom:.2f}\n"
        informe_str += f"Valor total:           $ {self.valorTotal:.2f}\n"
        informe_str += "Tipos de productos: \n"

        tipos = {}
        for producto in self.productos:
            if producto.tipo not in tipos:
                tipos[producto.tipo] = {'unidades': 0, 'precio_total': 0}
                tipos[producto.tipo]['unidades'] += producto.cantidad
                tipos[producto.tipo]['precio_total'] += producto.valorTotal()

        for tipo, info in tipos.items():
            precio_prom_tipo = info['precio_total'] / info['unidades'] if info['unidades'] > 0 else 0
            informe_str += f"  - {tipo}              :  {info['unidades']}u x $ {precio_prom_tipo:.2f}\n"

        informe_str += "Ofertas:\n"
        for oferta in self.ofertas:
            informe_str += f"  - {oferta.descripcion}\n"

        return informe_str
    

In [6]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [7]:
class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = nombre
        self._cuit = cuit 
    
    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, valor):
        if not valor.strip(): 
            print(f"el nombre no puede estar vacío")
        else:
            self._nombre = valor

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, valor):
        cuit = "XX-XXXXXXXX-X"
        if not valor.count('-') == 2:
            print(f"el formato del CUIT '{valor}' es incorrecto. el formato debe ser: {cuit}")
        elif not valor.replace("-","").isdigit():
            print(f"el formato {valor} es incorrecto. el formato debe ser: {self.cuit}")
        else:
            self._cuit = valor


In [8]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

el nombre no puede estar vacío
el formato del CUIT '1234567890123' es incorrecto. el formato debe ser: XX-XXXXXXXX-X
el formato CC-12345678-1 es incorrecto. el formato debe ser: 20-12345678-1
Prueba pasada exitosamente!


In [9]:
class Factura:
    ultimo_numero = 100 

    @classmethod
    def ultimaFactura(cls, numero):
        cls.ultimo_numero = numero

    @classmethod
    def nuevoNumero(cls):
        cls.ultimo_numero += 1
        return cls.ultimo_numero

    def __init__(self, catalogo, cliente):
        self.numero = self.nuevoNumero()
        self.fecha = "05/10/24"  
        self.cliente = cliente
        self.catalogo = catalogo
        self.productos = []  

    def agregar(self, producto, cantidad):
        for i, (prod, cant) in enumerate(self.productos):
            if prod.codigo == producto.codigo:
                self.productos[i] = (prod, cant + cantidad)
                break
        else:
            self.productos.append((producto, cantidad))
        producto.cantidad -= cantidad

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(cantidad for _, cantidad in self.productos)

    @property
    def subtotal(self):
        subtotal_general = sum(producto.precio * cantidad for producto, cantidad in self.productos)
        return subtotal_general

    @property
    def descuentos(self):
        total_descuentos = 0
        for producto, cantidad in self.productos:
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            total_descuentos += descuento
        return total_descuentos

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def imprimir(self):
        detalles = []
        for producto, cantidad in self.productos:
            precio = producto.precio
            subtotal = precio * cantidad
            detalles.append(f"- {cantidad} {producto.nombre}            x ${precio:.2f} = ${subtotal:.2f}")
           
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            descripcion_oferta = self.catalogo.obtenerDescripcionOferta(producto, cantidad)
            if descuento > 0:
                detalles.append(f"      {descripcion_oferta}                             - ${descuento:.2f}")

        factura_texto = f"Factura: {self.numero}\n"
        factura_texto += f"Fecha  : {self.fecha}\n"
        factura_texto += f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n\n"
        factura_texto += "\n".join(detalles) + "\n"
        factura_texto += f"\n                                             Subtotal:   ${self.subtotal:.2f}\n"
        factura_texto += f"                                             Descuentos: ${self.descuentos:.2f}\n"
        factura_texto += "                                             -----------------------\n"
        factura_texto += f"                                             TOTAL:      ${self.total:.2f}\n"
        
        return factura_texto

In [10]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [11]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
